In [1]:
import json
import pandas as pd
from owlready2 import *
import re
import fasttext 
import numpy as np
from numpy import dot
from numpy.linalg import norm
import networkx as nx
import annoy

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [17]:
AITopLevelTopics = ['Artificial intelligence', 'Computer vision', 'Data mining',
                     'Data science', 'Machine learning', 'Natural language processing',
                     'Pattern recognition', 'Speech recognition']
with open("dblpPaperIDs.json", 'r') as f:
    paperList = json.load(f)
papersUnderConsideration = set(paperList)

In [ ]:
paperData = dict()
with open('dblp_AIpapers_v11.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        
        dataDict = dict()
        dataDict['title'] = data.get('title', '')
        references = list()
        for reference in data.get('references',[]):
            if reference in papersUnderConsideration:
                references.append(reference)
        dataDict['references'] = references
        dataDict['abstract'] = []
        if 'indexed_abstract' in data:
            dataDict['abstract'] = [w for w in data['indexed_abstract']['InvertedIndex'].keys() if len(w) > 1]                    
        dataDict['fos'] = data.get('fos',[])
        # paperData[paperID] = dataDict
        tmpDict = dataDict.copy()
        tmpDict['id'] = paperID
        
        paperID = data['id'] 
            
            


In [7]:
fasttextModel = fasttext.load_model('crawl-300d-2M-subword.bin')

In [19]:
with open('./data/dblp_AIpapers2Thresholded.json', 'r') as file:
    with open('./data/dblpAbstract_2Thresholded_FT_Embeddings.json', 'w') as outfile:
        for line in file:
            data = json.loads(line)
            paperID = data['id'] 
            abstractWordList = data.get('abstract',[])
            abstractString = ' '.join(word for word in abstractWordList)
            abstractString = abstractString.replace('\n', ' ').replace('\r', '')
            embedding = fasttextModel.get_sentence_vector(abstractString).tolist()    # while reading use np.asarray to convert to np array
            outDict = dict()
            outDict['id'] = paperID
            outDict['embedding'] = embedding
            json.dump(outDict, outfile)
            outfile.write('\n')

In [ ]:
!curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed" | tar -zxvC ./module/UnivTransformer

In [3]:
tf.enable_eager_execution()

In [4]:
module_url = "./module/UnivTrans" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
def embed(inputText):
    return model(inputText)

In [22]:
count = 0
with open('dblp_AIpapers_v11.json', 'r') as file:
    with open('dblpAbstractUSETransEmbeddings.json', 'w') as outfile:
        for line in file:
            data = json.loads(line)
            paperID = data['id'] 
            abstractWordList = data.get('abstract',[])
            abstractString = ' '.join(word for word in abstractWordList)
            abstractString = abstractString.replace('\n', ' ').replace('\r', '')
            embedding = embed([abstractString])[0].numpy()
            outDict = dict()
            outDict['id'] = paperID
            outDict['embedding'] = embedding.tolist()
            json.dump(outDict, outfile)
            outfile.write('\n')
            if (count % 10000 == 0):
                logger.info('count: {0}'.format(count))
            count += 1

INFO:root:count: 0
INFO:root:count: 10000
INFO:root:count: 20000
INFO:root:count: 30000
INFO:root:count: 40000
INFO:root:count: 50000
INFO:root:count: 60000
INFO:root:count: 70000
INFO:root:count: 80000
INFO:root:count: 90000
INFO:root:count: 100000
INFO:root:count: 110000
INFO:root:count: 120000
INFO:root:count: 130000
INFO:root:count: 140000
INFO:root:count: 150000
INFO:root:count: 160000
INFO:root:count: 170000
INFO:root:count: 180000
INFO:root:count: 190000
INFO:root:count: 200000
INFO:root:count: 210000
INFO:root:count: 220000
INFO:root:count: 230000
INFO:root:count: 240000
INFO:root:count: 250000
INFO:root:count: 260000
INFO:root:count: 270000
INFO:root:count: 280000
INFO:root:count: 290000
INFO:root:count: 300000
INFO:root:count: 310000
INFO:root:count: 320000
INFO:root:count: 330000
INFO:root:count: 340000
INFO:root:count: 350000
INFO:root:count: 360000
INFO:root:count: 370000
INFO:root:count: 380000
INFO:root:count: 390000
INFO:root:count: 400000
INFO:root:count: 410000
INFO:r

In [ ]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    abstractEmbeddings = session.run(embed(abstractList))
del abstractList[:] 

In [ ]:
with open('dblpAbstractUSETransEmbeddings.json', 'w') as outfile:
    for paperId, embedding in zip(paperIDList, abstractEmbeddings):
        outDict = dict()
        outDict['id'] = paperID
        outDict['embedding'] = embedding.tolist()
        json.dump(outDict, outfile)
        outfile.write('\n')